In [1]:
import requests
import sqlite3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

conn = sqlite3.connect('iranjib.db')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS headlines')
cursor.execute('''CREATE TABLE headlines(
    link TEXT PRIMARY KEY,
    title TEXT,
    date TEXT,
    views TEXT,
    summary TEXT,
    content TEXT
)''')
conn.commit()

target_date = 14030830
saved = 0

for page_num in range(50):
    url = f'https://www.iranjib.ir/showarchive/0/{page_num}/'
    print(f'\nصفحه {page_num + 1}: {url}')
    
    r = requests.get(url, headers={'user-agent': 'mozilla/5.0'})
    page_soup = BeautifulSoup(r.content, 'html.parser')
    
    links_to_scrape = []
    for a in page_soup.find_all('a', href=True):
        href = a['href']
        title_text = a.get_text(strip=True)
        if '/shownews/' in href and (('طلا' in title_text) or ('دلار' in title_text)):
            href = urljoin(url, href)
            links_to_scrape.append(href)
    
    if not links_to_scrape:
        print('  هیچ خبری پیدا نشد')
        break
    
    for link in links_to_scrape:
        time.sleep(0.5)
        r = requests.get(link, headers={'user-agent':'mozilla/5.0'})
        article_soup = BeautifulSoup(r.content, 'html.parser')
        
        title_tag = article_soup.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else ''
        
        summary_tag = article_soup.find('div', {'class': 'newssummary'})
        summary = summary_tag.get_text(strip=True) if summary_tag else ''
        
        matn = article_soup.find('div', {'class': 'matn'})
        content = '\n'.join(p.get_text(strip=True) for p in matn.find_all('p')) if matn else ''
        
        date = views = ''
        tables = article_soup.find_all('table')
        if len(tables) > 1:
            trs = tables[1].find_all('tr')
            if len(trs) > 1:
                tds = trs[1].find_all('td')
                if len(tds) > 2:
                    date = tds[1].get_text(strip=True)
                    views = tds[2].get_text(strip=True)
        
        if date:
            try:
                clean_date = int(date.replace('/', ''))
                if clean_date < target_date:
                    print(f'  🛑 توقف: {date}')
                    conn.close()
                    print(f'\n✅ {saved} خبر')
                    exit()
            except: pass
        
        try:
            cursor.execute('INSERT INTO headlines (link, title, date, views, summary, content) VALUES (?,?,?,?,?,?)',
                          (link, title, date, views, summary, content))
            conn.commit()
            saved += 1
            print(f'  ✓ {saved} | {title[:40]} | {date}')
        except sqlite3.IntegrityError:
            print(f'  تکراری: {title[:40]}')

conn.close()
print(f'\n✅ {saved} خبر')


صفحه 1: https://www.iranjib.ir/showarchive/0/0/
  ✓ 1 | قیمت جدید سکه و طلا امروز پنج شنبه 1404/ | پنجشنبه، ۳۰ بهمن ۱۴۰۴ - ۱۲:۳۱:۳۲
  ✓ 2 | بهای طلا امروز پنج شنبه در بازارهای جهان | پنجشنبه، ۳۰ بهمن ۱۴۰۴ - ۱۱:۴۲:۴۲
  ✓ 3 | تجربه قیمت دلار بعد از قطعنامه ۵۹۸ | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۲۲:۵۶:۳۳
  ✓ 4 | قیمت جدید سکه و طلا امروز چهارشنبه 1404/ | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۱۳:۰۲:۲۰
  ✓ 5 | بهای طلا امروز چهارشنبه در بازارهای جهان | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۰۹:۲۳:۰۱
  ✓ 6 | 4 مدل پیش‌بینی جدید از روند قیمت طلا | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۰۸:۳۶:۳۸

صفحه 2: https://www.iranjib.ir/showarchive/0/1/
  تکراری: قیمت جدید سکه و طلا امروز پنج شنبه 1404/
  تکراری: بهای طلا امروز پنج شنبه در بازارهای جهان
  تکراری: تجربه قیمت دلار بعد از قطعنامه ۵۹۸
  تکراری: قیمت جدید سکه و طلا امروز چهارشنبه 1404/
  تکراری: بهای طلا امروز چهارشنبه در بازارهای جهان
  تکراری: 4 مدل پیش‌بینی جدید از روند قیمت طلا

صفحه 3: https://www.iranjib.ir/showarchive/0/2/
  تکراری: قیمت جدید سکه و طلا امروز پنج شنبه 1404/
  تکراری

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import sqlite3
import time

conn = sqlite3.connect('iranjib_news.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS news_data")
cursor.execute("""
CREATE TABLE news_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    link TEXT UNIQUE,
    title TEXT,
    date TEXT,
    views TEXT,
    summary TEXT,
    content TEXT
)
""")
conn.commit()

print('راه اندازی...')
options = Options()
options.add_argument('--headless')
options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get('https://www.iranjib.ir/showarchive/0/0/')
    time.sleep(3)
    
    target_date = 14030830
    saved = 0
    stop = False
    
    for page in range(1, 150):
        if stop: break
        
        print(f'\nصفحه {page}')
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        box = soup.find('div', id='column5')
        if not box: break
        
        page_links = []
        for a in box.find_all('a', href=True):
            title_text = a.get_text(strip=True)
            if ('طلا' in title_text) or ('دلار' in title_text):
                link = 'https://www.iranjib.ir' + a['href'] if a['href'].startswith('/') else a['href']
                page_links.append((link, title_text))
        
        for link, title_text in page_links:
            print(f'  {title_text[:40]}')
            driver.get(link)
            time.sleep(1.5)
            
            ps = BeautifulSoup(driver.page_source, 'html.parser')
            h1 = ps.find('h1')
            title = h1.get_text(strip=True) if h1 else ''
            summ = ps.find('div', {'class': 'newssummary'})
            summary = summ.get_text(strip=True) if summ else ''
            matn = ps.find('div', {'class': 'matn'})
            content = '\n'.join(p.get_text(strip=True) for p in matn.find_all('p')) if matn else ''
            
            date = views = ''
            tables = ps.find_all('table')
            if len(tables) > 1:
                trs = tables[1].find_all('tr')
                if len(trs) > 1:
                    tds = trs[1].find_all('td')
                    if len(tds) > 2:
                        date = tds[1].get_text(strip=True)
                        views = tds[2].get_text(strip=True)
            
            if date:
                try:
                    clean_date = int(date.replace('/', ''))
                    if clean_date < target_date:
                        print(f'  🛑 توقف: {date}')
                        stop = True
                        break
                except: pass

            cursor.execute("INSERT OR IGNORE INTO news_data (link, title, date, views, summary, content) VALUES (?, ?, ?, ?, ?, ?)",
                          (link, title, date, views, summary, content))
            conn.commit()
            saved += 1
            print(f'  ✓ {saved} | {date} | {views}')
        
        if stop: break
        
        driver.get('https://www.iranjib.ir/showarchive/0/0/')
        time.sleep(2)
        
        try:
            buttons = driver.find_elements(By.CLASS_NAME, 'paginbutton')
            clicked = False
            for btn in buttons:
                if btn.text.strip() == str(page + 1):
                    driver.execute_script("arguments[0].click();", btn)
                    clicked = True
                    break
            
            if not clicked:
                numeric_buttons = [b for b in buttons if b.text.strip().isdigit()]
                if numeric_buttons:
                    last_num_btn = numeric_buttons[-1]
                    driver.execute_script("arguments[0].click();", last_num_btn)
                    clicked = True
            
            if not clicked: break
            time.sleep(4)
            
        except Exception as e:
            print(f'  خطا: {e}')
            break

finally:
    driver.quit()
    conn.commit()
    conn.close()
    print(f'\n✅ {saved} خبر')


راه اندازی...

صفحه 1
  تجربه قیمت دلار بعد از قطعنامه ۵۹۸
  ✓ 1 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۲۲:۵۶:۳۳ | ۵۱۰۴ بازدید
  قیمت جدید سکه و طلا امروز چهارشنبه 1404/
  ✓ 2 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۱۳:۰۲:۲۰ | ۲۱۹۷۹ بازدید
  بهای طلا امروز چهارشنبه در بازارهای جهان
  ✓ 3 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۰۹:۲۳:۰۱ | ۳۶۲۰ بازدید
  4 مدل پیش‌بینی جدید از روند قیمت طلا
  ✓ 4 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۰۸:۳۶:۳۸ | ۵۵۳۱ بازدید
  دلار رسمی چند؟
  ✓ 5 | سه شنبه، ۲۸ بهمن ۱۴۰۴ - ۱۹:۵۳:۳۰ | ۴۶۰۴ بازدید
  قیمت جدید سکه و طلا امروز سه شنبه 1404/1
  ✓ 6 | سه شنبه، ۲۸ بهمن ۱۴۰۴ - ۱۲:۵۷:۱۷ | ۳۱۰۰۰ بازدید

صفحه 2
  بهای طلا امروز سه شنبه در بازارهای جهانی
  ✓ 7 | سه شنبه، ۲۸ بهمن ۱۴۰۴ - ۰۸:۵۰:۳۰ | ۴۴۰۶ بازدید
  قیمت جدید سکه و طلا امروز دوشنبه 1404/11
  ✓ 8 | دوشنبه، ۲۷ بهمن ۱۴۰۴ - ۱۲:۵۸:۰۳ | ۶۲۵۵۶ بازدید
  بهای طلا امروز دوشنبه در بازارهای جهانی
  ✓ 9 | دوشنبه، ۲۷ بهمن ۱۴۰۴ - ۰۹:۱۳:۰۵ | ۵۰۷۹ بازدید

صفحه 3
  قیمت دلار در مرکز مبادلات امروز
  ✓ 10 | یکشنبه، ۲۶ بهمن ۱۴۰۴ - ۲۰:۳۸:۳۷ | ۳۸۸۲ بازدید
  قیمت جدید سکه و طلا ا

KeyboardInterrupt: 

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import sqlite3
import time

conn = sqlite3.connect('iranjib_news.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS news_data")
cursor.execute("""
CREATE TABLE news_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    link TEXT UNIQUE,
    title TEXT,
    date TEXT,
    views TEXT,
    summary TEXT,
    content TEXT
)
""")
conn.commit()

print('راه اندازی...')
options = Options()
options.add_argument('--headless')
options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get('https://www.iranjib.ir/showarchive/0/0/')
    time.sleep(3)
    
    target_date = 14030830
    saved = 0
    stop = False
    
    for page in range(1, 150):
        if stop: break
        
        print(f'\nصفحه {page}')
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        box = soup.find('div', id='column5')
        if not box: break
        
        links = box.find_all('a', href=True)
        
        for a in links:
            title_text = a.get_text(strip=True)
            if ('طلا' not in title_text) and ('دلار' not in title_text):
                continue
            
            link = 'https://www.iranjib.ir' + a['href'] if a['href'].startswith('/') else a['href']
            
            print(f'  {title_text[:40]}')
            driver.get(link)
            time.sleep(1.5)
            
            ps = BeautifulSoup(driver.page_source, 'html.parser')
            h1 = ps.find('h1')
            title = h1.get_text(strip=True) if h1 else ''
            summ = ps.find('div', {'class': 'newssummary'})
            summary = summ.get_text(strip=True) if summ else ''
            matn = ps.find('div', {'class': 'matn'})
            content = '\n'.join(p.get_text(strip=True) for p in matn.find_all('p')) if matn else ''
            
            date = views = ''
            tables = ps.find_all('table')
            if len(tables) > 1:
                trs = tables[1].find_all('tr')
                if len(trs) > 1:
                    tds = trs[1].find_all('td')
                    if len(tds) > 2:
                        date = tds[1].get_text(strip=True)
                        views = tds[2].get_text(strip=True)
            
            if date:
                try:
                    clean_date = int(date.replace('/', ''))
                    if clean_date < target_date:
                        print(f'  🛑 توقف: {date}')
                        stop = True
                        break
                except: pass

            cursor.execute("INSERT OR IGNORE INTO news_data (link, title, date, views, summary, content) VALUES (?, ?, ?, ?, ?, ?)",
                          (link, title, date, views, summary, content))
            conn.commit()
            saved += 1
            print(f'  ✓ {saved} | {date} | {views}')
            
            driver.back()
            time.sleep(2)
        
        if stop: break
        
        try:
            buttons = driver.find_elements(By.CLASS_NAME, 'paginbutton')
            clicked = False
            for btn in buttons:
                if btn.text.strip() == str(page + 1):
                    driver.execute_script("arguments[0].click();", btn)
                    clicked = True
                    break
            
            if not clicked:
                numeric_buttons = [b for b in buttons if b.text.strip().isdigit()]
                if numeric_buttons:
                    last_num_btn = numeric_buttons[-1]
                    driver.execute_script("arguments[0].click();", last_num_btn)
                    clicked = True
                    time.sleep(2)
            
            if not clicked: break
            time.sleep(4)
            
        except Exception as e:
            print(f'  خطا: {e}')
            break

finally:
    driver.quit()
    conn.commit()
    conn.close()
    print(f'\n✅ {saved} خبر')


راه اندازی...

صفحه 1
  تجربه قیمت دلار بعد از قطعنامه ۵۹۸
  ✓ 1 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۲۲:۵۶:۳۳ | ۴۰۳۲ بازدید
  قیمت جدید سکه و طلا امروز چهارشنبه 1404/
  ✓ 2 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۱۳:۰۲:۲۰ | ۲۱۶۶۱ بازدید
  بهای طلا امروز چهارشنبه در بازارهای جهان
  ✓ 3 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۰۹:۲۳:۰۱ | ۳۵۷۶ بازدید
  4 مدل پیش‌بینی جدید از روند قیمت طلا
  ✓ 4 | چهارشنبه، ۲۹ بهمن ۱۴۰۴ - ۰۸:۳۶:۳۸ | ۵۵۰۰ بازدید
  دلار رسمی چند؟
  ✓ 5 | سه شنبه، ۲۸ بهمن ۱۴۰۴ - ۱۹:۵۳:۳۰ | ۴۵۹۸ بازدید
  قیمت جدید سکه و طلا امروز سه شنبه 1404/1
  ✓ 6 | سه شنبه، ۲۸ بهمن ۱۴۰۴ - ۱۲:۵۷:۱۷ | ۳۰۹۹۵ بازدید

صفحه 2
  بهای طلا امروز سه شنبه در بازارهای جهانی
  ✓ 7 | سه شنبه، ۲۸ بهمن ۱۴۰۴ - ۰۸:۵۰:۳۰ | ۴۴۰۱ بازدید
  قیمت جدید سکه و طلا امروز دوشنبه 1404/11
  ✓ 8 | دوشنبه، ۲۷ بهمن ۱۴۰۴ - ۱۲:۵۸:۰۳ | ۶۲۵۳۰ بازدید
  بهای طلا امروز دوشنبه در بازارهای جهانی
  ✓ 9 | دوشنبه، ۲۷ بهمن ۱۴۰۴ - ۰۹:۱۳:۰۵ | ۵۰۷۸ بازدید

صفحه 3
  قیمت دلار در مرکز مبادلات امروز
  ✓ 10 | یکشنبه، ۲۶ بهمن ۱۴۰۴ - ۲۰:۳۸:۳۷ | ۳۸۸۱ بازدید
  قیمت جدید سکه و طلا ا

KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import sqlite3
import time
import re

conn = sqlite3.connect('iranjib_news.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS news_data")
cursor.execute("""
CREATE TABLE news_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    link TEXT UNIQUE,
    title TEXT,
    date TEXT,
    views TEXT,
    summary TEXT,
    content TEXT
)
""")

print('راه اندازی...')
options = Options()
options.add_argument('--headless')
options.add_argument('--window-size=1920,1080') # برای اطمینان از دیده شدن دکمه‌ها
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get('https://www.iranjib.ir/showarchive/0/0/')
    time.sleep(3)
    
    target_date = 14030830
    saved = 0
    stop = False
    
    for page in range(1, 150): # تا صفحه ۱۵۰ برود که به آبان برسد
        if stop: break
        
        print(f'\nصفحه {page}')
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        box = soup.find('div', id='column5')
        if not box: break
        
        links = box.find_all('a', href=True)
        
        for a in links:
            title_text = a.get_text(strip=True)
            if ('طلا' not in title_text) and ('دلار' not in title_text) and ('سکه' not in title_text):
                continue
            
            link = 'https://www.iranjib.ir' + a['href'] if a['href'].startswith('/') else a['href']
            
            # استخراج محتوای خبر
            print(f'   {title_text[:40]}')
            driver.get(link)
            time.sleep(1.5)
            
            ps = BeautifulSoup(driver.page_source, 'html.parser')
            h1 = ps.find('h1')
            title = h1.get_text(strip=True) if h1 else ''
            summ = ps.find('div', {'class': 'newssummary'})
            summary = summ.get_text(strip=True) if summ else ''
            matn = ps.find('div', {'class': 'matn'})
            content = '\n'.join(p.get_text(strip=True) for p in matn.find_all('p')) if matn else ''
            
            date = views = ''
            tables = ps.find_all('table')
            if len(tables) > 1:
                tds = tables[1].find_all('td')
                if len(tds) > 2:
                    date = tds[1].get_text(strip=True)
                    views = tds[2].get_text(strip=True)
            
            # چک تاریخ برای توقف
            if date:
                try:
                    clean_date = int(date.replace('/', ''))
                    if clean_date < target_date:
                        print(f'   🛑 توقف در تاریخ: {date}')
                        stop = True
                        break
                except: pass

            cursor.execute("INSERT OR IGNORE INTO news_data (link, title, date, views, summary, content) VALUES (?, ?, ?, ?, ?, ?)",
                          (link, title, date, views, summary, content))
            saved += 1
            print(f'   ✓ {saved}')
            
            driver.back()
            time.sleep(2)
        
        if stop: break
        
        # --- بخش اصلاح شده برای رد کردن صفحه 8 و نقطه چین ---
        try:
            buttons = driver.find_elements(By.CLASS_NAME, 'paginbutton')
            clicked = False
            
            # ۱. اول بگرد دنبال شماره صفحه دقیق (مثلاً ۹)
            for btn in buttons:
                if btn.text.strip() == str(page + 1):
                    driver.execute_script("arguments[0].click();", btn)
                    clicked = True
                    break
            
            # ۲. اگر شماره صفحه پیدا نشد (گیر کردن در ۸)، روی آخرین شماره موجود کلیک کن
            if not clicked:
                # فیلتر کردن دکمه‌هایی که فقط عدد هستند
                numeric_buttons = [b for b in buttons if b.text.strip().isdigit()]
                if numeric_buttons:
                    last_num_btn = numeric_buttons[-1]
                    print(f"   🔄 دکمه {page+1} پیدا نشد، کلیک روی آخرین دکمه موجود ({last_num_btn.text}) برای باز شدن لیست...")
                    driver.execute_script("arguments[0].click();", last_num_btn)
                    clicked = True
                    time.sleep(2) # صبر برای جابجایی اعداد
            
            if not clicked:
                print('   ❌ پایان دکمه‌ها')
                break
                
            conn.commit()
            time.sleep(4) # صبر برای لود شدن لیست جدید اخبار
            
        except Exception as e:
            print(f'   خطا در صفحه زدن: {e}')
            break

finally:
    driver.quit()
    conn.commit()
    conn.close()
    print(f'\n✅ مجموعاً {saved} خبر ذخیره شد.')

راه اندازی...

صفحه 1
   تجربه قیمت دلار بعد از قطعنامه ۵۹۸
   ✓ 1
   قیمت جدید سکه و طلا امروز چهارشنبه 1404/
   ✓ 2
   بهای طلا امروز چهارشنبه در بازارهای جهان
   ✓ 3
   4 مدل پیش‌بینی جدید از روند قیمت طلا
   ✓ 4
   دلار رسمی چند؟
   ✓ 5
   قیمت جدید سکه و طلا امروز سه شنبه 1404/1
   ✓ 6

صفحه 2
   بهای طلا امروز سه شنبه در بازارهای جهانی
   ✓ 7
   قیمت جدید سکه و طلا امروز دوشنبه 1404/11
   ✓ 8
   بهای طلا امروز دوشنبه در بازارهای جهانی
   ✓ 9

صفحه 3
   قیمت دلار در مرکز مبادلات امروز
   ✓ 10
   قیمت جدید سکه و طلا امروز یکشنبه 1404/11
   ✓ 11
   پذیره‌نویسی سی‌ویکمین صندوق طلا از دوشنب
   ✓ 12
   نحوه قیمت‌گذاری طلا در بازار جهانی
   ✓ 13
   قیمت دلار در مرکز مبادلات امروز
   ✓ 14

صفحه 4
   قیمت جدید سکه و طلا امروز شنبه 1404/11/2
   ✓ 15
   تغییرات قیمت طلا در بازارهای جهانی
   ✓ 16
   زیان ۱.۲ میلیارد دلاری بنز ناشی از هزینه
   ✓ 17
   قیمت جدید سکه و طلا امروز پنج شنبه 1404/
   ✓ 18
   بهای طلا امروز پنج شنبه در بازارهای جهان
   ✓ 19
   ضربه تازه نرخ بیکاری آمریکا به بازار طل

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import sqlite3
import time
import re

conn = sqlite3.connect('iranjib_news.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS news_data")
cursor.execute("""
CREATE TABLE news_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    link TEXT UNIQUE,
    title TEXT,
    date TEXT,
    views TEXT,
    summary TEXT,
    content TEXT
)
""")

print('راه اندازی...')
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    driver.get('https://www.iranjib.ir/showarchive/0/0/')
    time.sleep(3)
    
    target_date = 14040830
    saved = 0
    stop = False
    
    for page in range(1, 51):
        if stop:
            break
        
        print(f'\nصفحه {page}')
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        box = soup.find('div', id='column5')
        if not box:
            print('بخش اخبار پیدا نشد')
            break
        
        links = box.find_all('a', href=True)
        
        for a in links:
            title = a.get_text(strip=True)
            
            if ('طلا' not in title) and ('دلار' not in title):
                continue
            
            link = 'https://www.iranjib.ir' + a['href'] if a['href'].startswith('/') else a['href']
            
            m = re.search(r'(\d{4})-(\d{1,2})-(\d{1,2})', link)
            if m:
                y, mn, d = map(int, m.groups())
                if (y * 10000 + mn * 100 + d) < target_date:
                    print(f'  توقف: تاریخ {y}/{mn:02d}/{d:02d} قبل از 30 آبان')
                    stop = True
                    break
            
            print(f'  {title[:40]}')
            driver.get(link)
            time.sleep(1)
            
            ps = BeautifulSoup(driver.page_source, 'html.parser')
            
            h1 = ps.find('h1')
            title = h1.get_text(strip=True) if h1 else ''
            
            summ = ps.find('div', {'class': 'newssummary'})
            summary = summ.get_text(strip=True) if summ else ''
            
            matn = ps.find('div', {'class': 'matn'})
            content = '\n'.join(p.get_text(strip=True) for p in matn.find_all('p')) if matn else ''
            
            date = views = ''
            tables = ps.find_all('table')
            if len(tables) > 1:
                trs = tables[1].find_all('tr')
                if len(trs) > 1:
                    tds = trs[1].find_all('td')
                    if len(tds) > 2:
                        date = tds[1].get_text(strip=True)
                        views = tds[2].get_text(strip=True)
            
            cursor.execute("INSERT OR IGNORE INTO news_data (link, title, date, views, summary, content) VALUES (?, ?, ?, ?, ?, ?)",
                          (link, title, date, views, summary, content))
            saved += 1
            print(f'  ✓ {saved}')
            
            driver.back()
            time.sleep(1)
        
        if stop:
            break
        
        try:
            buttons = driver.find_elements(By.CLASS_NAME, 'paginbutton')
            clicked = False
            for btn in buttons:
                if btn.text == str(page + 1):
                    btn.click()
                    conn.commit()
                    clicked = True
                    break
            if not clicked:
                print(f'دکمه صفحه {page + 1} پیدا نشد')
                break
        except Exception as e:
            print(f'خطا در صفحهبندی: {e}')
            break

finally:
    driver.quit()
    conn.commit()
    conn.close()
    print(f'\n✅ {saved} خبر ذخیره شد')


راه اندازی...

صفحه 1
  قیمت جدید سکه و طلا امروز چهارشنبه 1404/
  ✓ 1
  بهای طلا امروز چهارشنبه در بازارهای جهان
  ✓ 2
  4 مدل پیش‌بینی جدید از روند قیمت طلا
  ✓ 3
  دلار رسمی چند؟
  ✓ 4
  قیمت جدید سکه و طلا امروز سه شنبه 1404/1
  ✓ 5

صفحه 2
  بهای طلا امروز سه شنبه در بازارهای جهانی
  ✓ 6
  قیمت جدید سکه و طلا امروز دوشنبه 1404/11
  ✓ 7
  بهای طلا امروز دوشنبه در بازارهای جهانی
  ✓ 8

صفحه 3
  قیمت دلار در مرکز مبادلات امروز
  ✓ 9
  قیمت جدید سکه و طلا امروز یکشنبه 1404/11
  ✓ 10
  پذیره‌نویسی سی‌ویکمین صندوق طلا از دوشنب
  ✓ 11
  نحوه قیمت‌گذاری طلا در بازار جهانی
  ✓ 12
  قیمت دلار در مرکز مبادلات امروز
  ✓ 13

صفحه 4
  قیمت جدید سکه و طلا امروز شنبه 1404/11/2
  ✓ 14
  تغییرات قیمت طلا در بازارهای جهانی
  ✓ 15
  زیان ۱.۲ میلیارد دلاری بنز ناشی از هزینه
  ✓ 16
  قیمت جدید سکه و طلا امروز پنج شنبه 1404/
  ✓ 17
  بهای طلا امروز پنج شنبه در بازارهای جهان
  ✓ 18
  ضربه تازه نرخ بیکاری آمریکا به بازار طلا
  ✓ 19

صفحه 5
  بهای طلا امروز چهارشنبه در بازارهای جهان
  ✓ 20
  هشدار رئیس اتح

In [ ]:
for id, page_url in enumerate(new_urls):
page_response = requests.get(page_url)
page_soup = BeautifulSoup(page_response.text, 'html.parser')
page_title = page_soup.find('h1').get_text()
under_title = page_soup.find('div',{'class':'newssummary'}).get_text()
matn_page = page_soup.find('div',{'class':'matn'})
paragraphs = matn_page.find_all('p')
just_matn = ''
for p in paragraphs:
just_matn += p.get_text() +'\n'
table_page = page_soup.find_all('table')[1]
tr_of_table = table_page.find_all('tr')[1]
date = tr_of_table.find_all('td')[1].get_text()
views = tr_of_table.find_all('td')[2].get_text()
break